In [2]:
import pandas as pd 
import numpy as np
import requests 
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import scrapy
from scrapy.crawler import CrawlerProcess
import plotly.express as px
import logging
import os 

In [3]:
top_35_cities=["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
"Chateau du Haut Koenigsbourg","Colmar", "Eguisheim", "Besancon", "Dijon","Annecy", "Grenoble", "Lyon", "Gorges du Verdon",
"Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes",
"Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne",
"La Rochelle"]

In [4]:
df= pd.DataFrame()

for city in tqdm(top_35_cities):
    url="https://nominatim.openstreetmap.org/search?q="+city+"&format=json&countrycodes=fr&limit=1"
    result=requests.get(url)
    dict=result.json()[0]
    dict["city"]=city
    df=df.append(dict,ignore_index=True)

100%|██████████| 35/35 [00:17<00:00,  2.03it/s]


In [5]:
df.dtypes

place_id        float64
licence          object
osm_type         object
osm_id          float64
boundingbox      object
lat              object
lon              object
display_name     object
class            object
type             object
importance      float64
icon             object
city             object
dtype: object

In [6]:
df.head()

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance,icon,city
0,258697296.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,376823.0,"[48.6119741, 48.637031, -1.5495487, -1.5094805]",48.6355232,-1.5102571,"Le Mont-Saint-Michel, Avranches, Manche, Norma...",boundary,administrative,0.851274,https://nominatim.openstreetmap.org/ui/mapicon...,Mont Saint Michel
1,257985771.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,905534.0,"[48.5979853, 48.6949736, -2.0768518, -1.9367259]",48.649518,-2.0260409,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",boundary,administrative,0.676467,https://nominatim.openstreetmap.org/ui/mapicon...,St Malo
2,257654882.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,145776.0,"[49.2608124, 49.2934736, -0.7275671, -0.6757378]",49.2764624,-0.7024738,"Bayeux, Calvados, Normandie, France métropolit...",boundary,administrative,0.682700,https://nominatim.openstreetmap.org/ui/mapicon...,Bayeux
3,256418097.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492.0,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",boundary,administrative,0.822333,https://nominatim.openstreetmap.org/ui/mapicon...,Le Havre
4,135340.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686587.0,"[49.2804591, 49.6004591, 0.9339658, 1.2539658]",49.4404591,1.0939658,"Rouen, Seine-Maritime, Normandie, France métro...",place,city,0.750073,https://nominatim.openstreetmap.org/ui/mapicon...,Rouen


In [7]:
df2= pd.DataFrame()

for index, row in tqdm(df.iterrows()):
    lat=row["lat"]
    lon=row["lon"]
    city=row["city"]
    url="https://api.openweathermap.org/data/2.5/onecall?lat="+lat+"&lon="+lon+"&exclude=current,minutely,hourly&units=metric&appid=b4a7f944eb178713c5d742788c44e014"
    result=requests.get(url)
    dict_=result.json()
    df3= pd.DataFrame()
    
    for i in range(0,8):
        
        number=str(i)
        
        t_min=dict_["daily"][i]["temp"]["min"]
        t_max=dict_["daily"][i]["temp"]["max"]
        t_day=dict_["daily"][i]["temp"]["day"]
        t_eve=dict_["daily"][i]["temp"]["eve"]
        t_night=dict_["daily"][i]["temp"]["night"]
        pressure=dict_["daily"][i]["pressure"]
        humid=dict_["daily"][i]["humidity"]
        dew_point=dict_["daily"][i]["dew_point"]
        wind_speed=dict_["daily"][i]["wind_speed"]
        wind_deg=dict_["daily"][i]["wind_deg"]
        wind_gust=dict_["daily"][i]["wind_gust"]
        clouds=dict_["daily"][i]["clouds"]
        
        keys=["t_min_"+number,"t_max_"+number,"t_day_"+number,"t_eve_"+number,"t_night_"+number,
              "pressure_"+number,"humid_"+number,"dew_point_"+number,"wind_speed_"+number,"wind_deg_"+number,
              "wind_gust_"+number,"clouds"+number]
        values=[t_min,t_max,t_day,t_eve,t_night,pressure,humid,dew_point,wind_speed,wind_deg,wind_gust,clouds]
        
        #dictionary=dict(zip(keys, values))
        #dictionary["city"]=city
        df_f = pd.DataFrame(values,index =keys)
        frames = [df3, df_f]
        df3=pd.concat(frames)
    
    df3=df3.T
    df2=df2.append(df3, ignore_index=True)
        

df2.head()
        

35it [00:09,  3.87it/s]


,t_min_0,t_max_0,t_day_0,t_eve_0,t_night_0,pressure_0,humid_0,dew_point_0,wind_speed_0,wind_deg_0,...,t_day_7,t_eve_7,t_night_7,pressure_7,humid_7,dew_point_7,wind_speed_7,wind_deg_7,wind_gust_7,clouds7
0,10.55,18.43,17.01,14.22,12.82,1021.0,65.0,10.15,3.80,119.0,...,11.78,8.44,9.26,1025.0,53.0,2.42,6.38,158.0,12.34,52.0
1,12.28,17.71,16.20,14.45,13.37,1020.0,67.0,10.03,4.25,140.0,...,11.95,10.72,10.43,1024.0,53.0,2.74,8.08,168.0,12.85,65.0
2,10.01,18.23,16.73,14.07,12.39,1021.0,67.0,10.60,3.90,155.0,...,11.39,7.27,7.56,1025.0,52.0,1.88,5.55,162.0,12.51,32.0
3,12.11,16.68,15.02,14.44,13.54,1021.0,71.0,9.73,4.20,161.0,...,10.47,9.74,8.80,1025.0,51.0,0.80,7.03,157.0,10.83,29.0
4,10.22,17.00,14.73,12.56,11.29,1022.0,65.0,7.76,2.51,130.0,...,10.64,6.51,5.32,1026.0,54.0,1.45,3.21,150.0,7.72,48.0


In [8]:
final_df=pd.concat([df, df2], axis=1)
final_df.head()

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,...,t_day_7,t_eve_7,t_night_7,pressure_7,humid_7,dew_point_7,wind_speed_7,wind_deg_7,wind_gust_7,clouds7
0,258697296.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,376823.0,"[48.6119741, 48.637031, -1.5495487, -1.5094805]",48.6355232,-1.5102571,"Le Mont-Saint-Michel, Avranches, Manche, Norma...",boundary,administrative,...,11.78,8.44,9.26,1025.0,53.0,2.42,6.38,158.0,12.34,52.0
1,257985771.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,905534.0,"[48.5979853, 48.6949736, -2.0768518, -1.9367259]",48.649518,-2.0260409,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",boundary,administrative,...,11.95,10.72,10.43,1024.0,53.0,2.74,8.08,168.0,12.85,65.0
2,257654882.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,145776.0,"[49.2608124, 49.2934736, -0.7275671, -0.6757378]",49.2764624,-0.7024738,"Bayeux, Calvados, Normandie, France métropolit...",boundary,administrative,...,11.39,7.27,7.56,1025.0,52.0,1.88,5.55,162.0,12.51,32.0
3,256418097.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492.0,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",boundary,administrative,...,10.47,9.74,8.80,1025.0,51.0,0.80,7.03,157.0,10.83,29.0
4,135340.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686587.0,"[49.2804591, 49.6004591, 0.9339658, 1.2539658]",49.4404591,1.0939658,"Rouen, Seine-Maritime, Normandie, France métro...",place,city,...,10.64,6.51,5.32,1026.0,54.0,1.45,3.21,150.0,7.72,48.0


In [9]:
final_df['T_Average'] = final_df[['t_min_0', 't_max_0', 't_day_0', 't_eve_0', 't_night_0',
                            't_min_1', 't_max_1', 't_day_1', 't_eve_1', 't_night_1',
                           't_min_2', 't_max_2', 't_day_2', 't_eve_2', 't_night_2',
                            't_min_3', 't_max_3', 't_day_3', 't_eve_3', 't_night_3',
                            't_min_4', 't_max_4', 't_day_4', 't_eve_4', 't_night_4',
                            't_min_5', 't_max_5', 't_day_5', 't_eve_5', 't_night_5',
                            't_min_6', 't_max_6', 't_day_6', 't_eve_6', 't_night_6',
                            't_min_7', 't_max_7', 't_day_7', 't_eve_7', 't_night_7',                           
                           ]].mean(axis=1)

final_df['T_Average'] = final_df['T_Average'].astype(float)
final_df['lat'] = final_df['lat'].astype(float)
final_df['lon'] = final_df['lon'].astype(float)

final_df=final_df.sort_values(by='T_Average', ascending=False).reset_index(drop=True)
final_df.head()

,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,...,t_eve_7,t_night_7,pressure_7,humid_7,dew_point_7,wind_speed_7,wind_deg_7,wind_gust_7,clouds7,T_Average
0,258528870.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,18409.0,"[42.4832453, 42.5341349, 3.0495074, 3.0994238]",42.525050,3.083155,"Collioure, Céret, Pyrénées-Orientales, Occitan...",boundary,administrative,...,14.41,13.19,1020.0,47.0,6.01,10.26,330.0,16.16,1.0,17.52575
1,258706271.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,76469.0,"[43.1696228, 43.3910329, 5.2286312, 5.5324758]",43.296174,5.369953,"Marseille, Bouches-du-Rhône, Provence-Alpes-Cô...",boundary,administrative,...,12.88,12.47,1016.0,51.0,3.29,15.67,324.0,20.46,0.0,17.39275
2,258573465.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,76425.0,"[43.1884732, 43.2487231, 5.5063193, 5.5945199]",43.214036,5.539632,"Cassis, Marseille, Bouches-du-Rhône, Provence-...",boundary,administrative,...,13.02,12.73,1015.0,48.0,2.72,14.64,322.0,20.26,0.0,17.22500
3,258745823.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,166713.0,"[43.4498733, 43.5188846, -1.5048438, -1.417261]",43.493338,-1.475099,"Bayonne, Pyrénées-Atlantiques, Nouvelle-Aquita...",boundary,administrative,...,13.11,12.05,1023.0,70.0,10.45,1.77,307.0,3.38,65.0,16.86900
4,258573486.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1118915.0,"[43.0864271, 43.2278201, 6.2716142, 6.4514893]",43.150697,6.341928,"Bormes-les-Mimosas, Toulon, Var, Provence-Alpe...",boundary,administrative,...,14.24,13.93,1012.0,34.0,1.00,14.01,307.0,20.27,0.0,16.83700


In [10]:
final_df.to_excel("cities.xlsx")
final_df.to_csv("cities.csv")

In [11]:
fig = px.scatter_mapbox(final_df, lat="lat", lon="lon", color="T_Average", zoom =4,size="T_Average",size_max=10,
                        mapbox_style="open-street-map", color_continuous_scale = 'Bluered', range_color = [8,20.0],
                       )
fig.show()
